In [ ]:
import pandas as pd
import math
import os
import random

from tqdm.notebook import tqdm

from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp import DoublespaceLineCorpus

from konlpy.tag import Okt

In [ ]:
random.seed(42)

In [ ]:
train = pd.read_excel('./paper_labels_3_all_random_300_train.xlsx')
val = pd.read_excel('./paper_labels_3_all_random_300_val.xlsx')

In [ ]:
train

In [ ]:
val

In [ ]:
train_abst = train['abstract']
val_abst = val['abstract']

In [ ]:
train_abst

In [ ]:
val_abst

In [ ]:
train_sent = list(train_abst)
train_sent

In [ ]:
val_sent = list(val_abst)
val_sent

In [ ]:
sentences = train_sent + val_sent

In [ ]:
len(sentences)

In [ ]:
word_extractor = WordExtractor(min_frequency = 1, min_cohesion_forward = 0.0, min_right_branching_entropy = 0.0)

word_extractor.train(sentences)
word_extractor.save('./wordextractor_model_all_random_300')

words = word_extractor.extract()

In [ ]:
#words

In [ ]:
scores = word_extractor.word_scores()
cal_scores = {key:(scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) for key in scores.keys()}
tokenizer = LTokenizer(scores = cal_scores)

In [ ]:
ltokenized_data = []


f = open('./user_dic_all_random_300.txt', 'w', encoding='UTF8')
for sentence in tqdm(sentences):
    tokens = tokenizer.tokenize(sentence)
    for token in tokens:
        token = token+'\n'
        f.write(token)
        ltokenized_data.append(token)
        print(token)

f.close()

In [ ]:
len(ltokenized_data)

In [ ]:
okt = Okt()
tokenized_data = []


for sentence in tqdm(sentences):
    temp_X = okt.morphs(sentence)
    tokenized_data.append(temp_X)

In [ ]:
tokenized_data

# okt 사용자 사전 추가

In [ ]:
import os

In [ ]:
os.chdir('C:/Users/user/.conda/envs/mlc/lib/site-packages/konlpy/java')
os.getcwd() 
os.makedirs('./userdic')

In [ ]:
!jar xvf open-korean-text-2.1.0.jar

In [ ]:
!jar cvf open-korean-text-2.1.0.jar *

In [ ]:
"""
os.chdir('D:/Anaconda3/envs/han/lib/site-packages/konlpy/java')
os.getcwd() 
os.makedirs('./userdic')


!jar xvf ../open-korean-text-2.1.0.jar


with open(f"D:/Anaconda3/envs/han/lib/site-packages/konlpy/java/userdic/org/openkoreantext/processor/util/noun/names.txt", encoding='UTF8') as f:
    data = f.read()
    
    
for tokens in tqdm(ltokenized_data):
    for token in tokens:
        data += token + '\n'
        
        
with open("D:/Anaconda3/envs/han/lib/site-packages/konlpy/java/userdic/org/openkoreantext/processor/util/noun/names.txt", 'w', encoding='UTF8') as f:
    f.write(data)
    
    
!jar cvf ../open-korean-text-2.1.0.jar * 
"""

# 명사 추출 토크나이저

In [ ]:
"""
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(sentences) # list of str like

noun_scores = {noun:score.score for noun, score in nouns.items()}
cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0) for noun, score in noun_scores.items()}
combined_scores.update({subword:cohesion for subword, cohesion in cohesion_score.items() if not (subword in combined_scores)})

tokenizer = LTokenizer(scores=combined_scores)




noun_tokenized_data = []


for sentence in tqdm(sentences):
    tokens = tokenizer.tokenize(sentence)
    noun_tokenized_data.append(tokens)
    print(tokens)
"""

In [1]:
import pandas as pd
from konlpy.tag import Okt

In [3]:
okt = Okt()

#rae_v17_train_10
sentence1 = "본 논문에서는 최신의 기계 학습 모델을 사용하여 서울 아파트 거래 가격에 대한 자동 평가 모델을 개발하였다."

#rae_v17_train_5
sentence2 = "지능정보사회의 도래로 종래와는 차원이 다른 새로운 이슈들이 제기될 것으로 전망된다."

#rae_v17_train_7
sentence3 = "최근 빅데이터 분석과 인공지능을 활용한 정보서비스로 이용자의 편리성을 확대하고, 미래의 상황을 예측하는 등 새로운 가치 창출 시스템으로 지능형서비스가 등장하였다."

print(okt.morphs(sentence1))
print(okt.morphs(sentence2))
print(okt.morphs(sentence3))

['본', '논문에서는', '최신', '의', '기계', '학습', '모델을', '사용하여', '서울', '아파트', '거래', '가격에', '대한', '자동', '평가', '모델을', '개발', '하였다', '.']
['지능', '정보사회의', '도래', '로', '종래와는', '차원', '이', '다른', '새로운', '이슈들', '이', '제기', '될', '것으로', '전망', '된다', '.']
['최근', '빅데이터', '분석과', '인공지능을', '활용한', '정보', '서비스로', '이용자의', '편리성', '을', '확대', '하고', ',', '미래', '의', '상황을', '예측', '하는', '등', '새로운', '가치', '창출', '시스템으로', '지능', '형서비스가', '등장', '하였다', '.']
